In [10]:
import torch
from torch.utils.data import Dataset , DataLoader
import tiktoken
vocab_size=50257
output_dim=256

token_embedding_layer =torch.nn.Embedding(vocab_size,output_dim)

In [46]:
class GPTdatasetV1(Dataset):
    def __init__( self ,raw_text , tokenizer ,max_length ,stride):
        self.input_ids = []
        self.target_ids = []

        #tokenizer the entire text

        token_ids =tokenizer.encode(raw_text , allowed_special={"<endoftext>"})

        #use a sliding window to chunk the book into overlapping sequence of max_length

        for i in range( 0, len(token_ids) - max_length, stride):
            input_chunk =token_ids[i:i+max_length]
            target_chunk =token_ids[i+1:i+max_length +1]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunk))
    
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self,idx):
        return self.input_ids[idx], self.target_ids[idx]

In [55]:
from torch.utils.data import DataLoader

# Define the function to create the DataLoader
def create_dataloader_V1(raw_text, batch_size=4, max_length=256, stride=128, shuffle=True, drop_last=True, num_workers=0):
    
    # Initialize the tokenizer (assuming tiktoken is the correct tokenizer library)
    tokenizer = tiktoken.get_encoding("gpt2")

    # Create the dataset using the GPTdatasetV1 class
    dataset = GPTdatasetV1(raw_text, tokenizer, max_length, stride)

    # Create DataLoader (note: raw_text is not passed to DataLoader)
    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        drop_last=drop_last,
        num_workers=num_workers
    )
    
    return dataloader


In [56]:
with open("the-verdict.txt" , "r" , encoding="utf-8") as f:
    raw_text=f.read()
max_length=4
dataloader= create_dataloader_V1( raw_text , batch_size=8 ,max_length=max_length ,stride=max_length ,shuffle=False)
data_iter =iter(dataloader)
inputs,targets =next(data_iter)

In [57]:
print("TOKEN ID's:\n",inputs)
print("Input Shape:\n",inputs.shape)

TOKEN ID's:
 tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])
Input Shape:
 torch.Size([8, 4])


In [58]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [71]:
context_length =max_length
pos_embedding_layer =torch.nn.Embedding(context_length ,output_dim)
print(pos_embedding_layer)

Embedding(4, 256)


In [72]:

pos_embedding = pos_embedding_layer(torch.arange(max_length))
print(pos_embedding.shape)

torch.Size([4, 256])
